In [1]:
import CNN2Head_input2 as CNN2Head_input
import tensorflow as tf
import numpy as np
import BKNetStyle2 as BKNetStyle
from const import *
import cv2

from IPython.display import Image, display
from mtcnn.mtcnn import MTCNN
from matplotlib import pyplot as plt

In [2]:
import os
from skimage import io
import instaloader
from instaloader import *
import urllib.request
import pickle

In [4]:
def predict(PATH, post):
    PATH1 = PATH + post
    post = load_structure_from_file(L.context, PATH1)
    img = io.imread(post.url)
    #print(post.url)
    #urllib.request.urlretrieve(post.url, "/scratch/jamaalhay/PostsNew/00000001.jpg")
    #PATH = PATH + "00000001.jpg"
    #display(Image(filename = PATH, width=100, height=100))
    count_s = 0
    count_g = 0
    result = detector.detect_faces(img)
    face_position = result[0].get('box')
    x_coordinate = face_position[0]
    y_coordinate = face_position[1]
    w_coordinate = face_position[2]
    h_coordinate = face_position[3]
    img = img[y_coordinate:y_coordinate+h_coordinate, x_coordinate:x_coordinate+w_coordinate]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img,(48, 48))
    img = (img - 128) / 255.0
    T = np.zeros([48, 48, 1])
    T[:, :, 0] = img
    test_img = []
    test_img.append(T)
    test_img = np.asarray(test_img)
    for i in range(30):
        init = tf.global_variables_initializer()
        sess.run(init)
        predict_y_smile_conv = sess.run(y_smile_conv, feed_dict={x: test_img, phase_train: False, keep_prob: 1})
        predict_y_gender_conv = sess.run(y_gender_conv, feed_dict={x: test_img, phase_train: False, keep_prob: 1})
        predict_y_age_conv = sess.run(y_age_conv, feed_dict={x: test_img, phase_train: False, keep_prob: 1})
        smile_label = "No" if np.argmax(predict_y_smile_conv)==0 else "Yes"
        gender_label = "Female" if np.argmax(predict_y_gender_conv)==0 else "Male"
        if smile_label == "Yes":
            count_s += 1
        if count_s >= 15:
            smile_label = "Yes"
        else:
            smile_label = "No"
        if gender_label == "Male":
            count_g += 1
        if count_g >= 15:
            gender_label = "Male"
        else:
            gender_label = "Female"
    return smile_label, gender_label

In [5]:
x, y_, mask = BKNetStyle.Input()

y_smile_conv, y_gender_conv, y_age_conv, phase_train, keep_prob = BKNetStyle.BKNetModel(x)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:

smile_loss, gender_loss, age_loss, l2_loss, loss = BKNetStyle.selective_loss(y_smile_conv, y_gender_conv,
                                                                             y_age_conv, y_, mask)
sess = tf.InteractiveSession()
print('Restore model')
with tf.Session() as sess1:
    saver = tf.train.Saver()
    saver = tf.train.import_meta_graph('./save/current6/model-age101.ckpt.meta')
    saver.restore(sess1, './save/current6/model-age101.ckpt')
print('OK')


Restore model
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./save/current6/model-age101.ckpt
OK


In [7]:
detector = MTCNN()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [10]:
posts = os.listdir('/scratch/jamaalhay/PostsNew')


In [8]:
L = instaloader.Instaloader()

In [12]:
dictt = {}
count = 0
for post in posts:
    count += 1
    if count % 100 == 1:
        print(count)
    smile, gender = predict('/scratch/jamaalhay/PostsNew/', post)
    dictt[post[:-5]] = [smile, gender]

In [13]:
pickle.dump(dictt, open( "save_smile.p", "wb" ) )

1738166964353067210 ['Yes', 'Female']
